In [55]:
import pipeline
import pandas as pd
import importlib
importlib.reload(pipeline)

<module 'pipeline' from 'C:\\Google Drive\\Machine Learning\\ml-covid\\scripts\\pipeline.py'>

In [56]:
filepath = '../data/covid_df_protocol_3.pkl'
df = pd.read_pickle(filepath)

In [57]:
weeks_for_cv = [0,1,2,3,4,5,6]
n_weeks_prediction = 2
results = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    pipeline.sanity_check(df_train, df_test)
    #df_train = df_train.dropna()
    #df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Daily New Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Daily New Cases')
    dct = pipeline.train_and_evaluate(X_train, y_train, X_test, y_test)
    results['Until week: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-29
Success: Features match
Success: No NAs remain


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11450362200.126648, tolerance: 4971216.729031145
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
48                         Country_China -5361.164445
5                        StringencyIndex -5154.569630
0                              Intercept  2169.785394
203  H1_Public information campaigns_2.0  1807.948329
198                E1_Income support_2.0  1675.665983
181        C5_Close public transport_0.0 -1477.643076
199          E2_Debt/contract relief_0.0 -1281.721318
44                        Country_Canada -1176.620134
207                H2_Testing policy_3.0  1145.013971
172             C2_Workplace closing_3.0  1022.256151

Bias: 1227636.24
Mean squared error: 3550204.71
RSS: 5275604205.47
Variance score: 0.51

R2 score: 0.53

Ridge: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
0                              Intercept  38

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8864240634.810156, tolerance: 4407638.045617789
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
5                        StringencyIndex -4896.078564
48                         Country_China -4391.837488
0                              Intercept  2623.014870
200                E1_Income support_2.0  1863.681673
205  H1_Public information campaigns_2.0  1713.680376
183        C5_Close public transport_0.0 -1427.082308
201          E2_Debt/contract relief_0.0 -1371.889861
209                H2_Testing policy_3.0  1234.736490
44                        Country_Canada -1074.821847
141                  Country_South Korea -1038.946660

Bias: 1179719.40
Mean squared error: 2655660.63
RSS: 5226340120.05
Variance score: 0.50

R2 score: 0.55

Ridge: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
0                              Intercept  36

...

Lasso: Features with highest magnitude                                coefficients in absolute value
                           Feature  Coefficient
0                        Intercept  3669.279825
5                  StringencyIndex -3022.184075
199          E1_Income support_2.0  2284.304630
48                   Country_China -1823.774822
200    E2_Debt/contract relief_0.0 -1529.407348
209          H2_Testing policy_3.0  1373.480549
79                   Country_India  1299.964411
140            Country_South Korea -1141.856538
182  C5_Close public transport_0.0  -935.142557
8             EconomicSupportIndex  -920.342344

Bias: 877737.93
Mean squared error: 3415913.81
RSS: 6913809552.48
Variance score: 0.43

R2 score: 0.49

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  5871.967427
199        E1_Income support_2.0  2191.870561
5                Stringency

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3827016746.5701, tolerance: 1649416.3287543827
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  3601.709064
5                StringencyIndex -2296.998423
200        E1_Income support_2.0  2203.274237
210        H2_Testing policy_3.0  1508.451370
201  E2_Debt/contract relief_0.0 -1485.260396
80                 Country_India  1203.049110
141          Country_South Korea -1153.747476
49                 Country_China  -997.989592
198        E1_Income support_0.0  -888.552680
8           EconomicSupportIndex  -823.314635

Bias: 670615.67
Mean squared error: 4083572.75
RSS: 8318237686.73
Variance score: 0.40

R2 score: 0.42

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  6374.209750
200        E1_Income support_2.0  2247.553568
198        E1_Income support_0.0 -1790.745451
201

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2040590255.1160975, tolerance: 840830.0780238091
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  2490.551916
201        E1_Income support_2.0  1731.079290
211        H2_Testing policy_3.0  1246.042171
202  E2_Debt/contract relief_0.0 -1158.794707
5                StringencyIndex -1117.720113
142          Country_South Korea  -923.350637
87                 Country_Italy   704.028891
143                Country_Spain   701.976385
199        E1_Income support_0.0  -648.402821
81                 Country_India   647.730252

Bias: 383009.73
Mean squared error: 5066488.06
RSS: 10381234030.04
Variance score: 0.38

R2 score: 0.32

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  6005.365640
201        E1_Income support_2.0  2039.226468
199        E1_Income support_0.0 -1653.691965
20

In [58]:
metrics_r2 = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_r2.loc[i,key] = results[mkey][key][4]
metrics_r2

,Lasso,Ridge,Linear Regression
0,0.529045,0.538256,0.541861
1,0.554805,0.56438,0.567738
2,0.544722,0.557039,0.562171
3,0.520936,0.536062,0.540168
4,0.485592,0.499308,0.502111
5,0.419886,0.434213,0.435185
6,0.319501,0.343298,0.343906


In [59]:
metrics_mse = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_mse.loc[i,key] = results[mkey][key][1]
metrics_mse

,Lasso,Ridge,Linear Regression
0,3.5502e+06,3.48077e+06,3.4536e+06
1,2.65566e+06,2.59855e+06,2.57851e+06
2,2.61214e+06,2.54147e+06,2.51203e+06
3,2.99953e+06,2.90483e+06,2.87911e+06
4,3.41591e+06,3.32483e+06,3.30622e+06
5,4.08357e+06,3.98273e+06,3.97588e+06
6,5.06649e+06,4.88932e+06,4.88478e+06


## with Grid Search for Lasso and Ridge

In [60]:
weeks_for_cv = [0,1,2,3,4,5,6]
n_weeks_prediction = 2
results_grid = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    pipeline.sanity_check(df_train, df_test)
    #df_train = df_train.dropna()
    #df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')
    dct = pipeline.train_and_evaluate_w_grid(X_train, y_train, X_test, y_test)
    results_grid['Until week: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-29
Success: Features match
Success: No NAs remain


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12726093503447.85, tolerance: 4685249832.446781
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12862069322488.672, tolerance: 4604265618.223251
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12599971566052.12, tolerance: 4599490281.589481
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality 

{'mean_fit_time': array([2.30172725, 2.3564045 , 2.25027347, 2.2892343 , 2.37999988]), 'std_fit_time': array([0.06501662, 0.12712053, 0.03408474, 0.08134468, 0.11269952]), 'mean_score_time': array([0.00735292, 0.00736341, 0.00831676, 0.00815954, 0.00877652]), 'std_score_time': array([0.00097781, 0.00050993, 0.00123555, 0.00072778, 0.00074549]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-2.54575013, -2.48858474, -2.46806995, -2.46483614, -2.45195115]), 'split1_test_score': array([-0.08001423, -0.09263021, -0.10402982, -0.09967808, -0.09697123]), 'split2_test_score': array([-37.88553826, -28.9684314 , -21.328622  , -14.93023786,
        -9.78823959]), 'split3_test_score': array([-2.12900363, -1.97978972, -1.89111314, -1.83398005, -1.77365426]), 'spl

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8849520047414.643, tolerance: 3170478422.125867
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8912436787547.947, tolerance: 3094368474.6145883
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8706965532890.746, tolerance: 3096297560.8648667
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality

{'mean_fit_time': array([2.44546371, 2.42881241, 2.49598107, 2.4958714 , 2.59871531]), 'std_fit_time': array([0.11138286, 0.01007304, 0.04011952, 0.03480785, 0.07502963]), 'mean_score_time': array([0.00862646, 0.0090095 , 0.00903444, 0.008002  , 0.00977669]), 'std_score_time': array([0.00101549, 0.00103861, 0.00090044, 0.00107197, 0.00116078]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-2.04642698, -2.03845149, -2.02909953, -2.02060536, -2.02127656]), 'split1_test_score': array([0.0351389 , 0.02650834, 0.0219254 , 0.02306484, 0.02407996]), 'split2_test_score': array([-17.77646608, -11.26432211,  -6.32712803,  -2.95856659,
        -1.16530183]), 'split3_test_score': array([-3.26653053, -3.22308771, -3.20040668, -3.20263442, -3.19194707]), 'split4_t

...

{'mean_fit_time': array([2.6302906 , 2.68729167, 2.54852319, 2.4774889 , 2.63321199]), 'std_fit_time': array([0.09987116, 0.13390655, 0.1561092 , 0.04122816, 0.06913204]), 'mean_score_time': array([0.01015792, 0.00957527, 0.00932755, 0.00937471, 0.0091888 ]), 'std_score_time': array([0.00172672, 0.00162121, 0.00083781, 0.0010175 , 0.00076366]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-0.2888484 , -0.28317159, -0.27925966, -0.27041448, -0.25966014]), 'split1_test_score': array([0.13235399, 0.13629754, 0.13452865, 0.13465497, 0.13504402]), 'split2_test_score': array([-952.38740232, -737.68917669, -550.29606037, -390.3024759 ,
       -257.85427019]), 'split3_test_score': array([-15.97261646, -15.89828448, -15.81704907, -15.72822818,
       -15.6092

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 679264028565.058, tolerance: 220577347.76082414
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 695805555545.3557, tolerance: 238725899.83855957
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 646483968702.2106, tolerance: 229526311.60403845
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality

{'mean_fit_time': array([2.65063868, 2.48974671, 2.43056436, 2.54102688, 2.45030813]), 'std_fit_time': array([0.17396626, 0.11362801, 0.11442608, 0.27160941, 0.11361535]), 'mean_score_time': array([0.00857697, 0.00856185, 0.00921197, 0.00908298, 0.00937486]), 'std_score_time': array([0.00079792, 0.00146848, 0.00121527, 0.00135211, 0.00149312]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([0.20191663, 0.21420096, 0.22020369, 0.2249503 , 0.22948802]), 'split1_test_score': array([0.04082807, 0.03701767, 0.03554229, 0.03781321, 0.03786972]), 'split2_test_score': array([-5231.42803452, -4565.56973341, -3946.18396316, -3371.10312418,
       -2841.33150616]), 'split3_test_score': array([-18.71607579, -18.58877469, -18.46438072, -18.33148291,
       -18.2343

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 185374529012.93427, tolerance: 60489037.085545205
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 212221144289.9995, tolerance: 86156056.45739678
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 187550524758.89877, tolerance: 79798507.65063699
  positive)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

{'mean_fit_time': array([2.35051937, 2.39271855, 2.87573686, 3.26344199, 3.07096148]), 'std_fit_time': array([0.08456577, 0.11941271, 1.02610989, 0.80950694, 0.41468339]), 'mean_score_time': array([0.0090127 , 0.00797815, 0.00858412, 0.00898199, 0.00997362]), 'std_score_time': array([1.19712894e-03, 1.70864800e-06, 1.02530147e-03, 6.38772887e-04,
       1.09214478e-03]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([0.27542837, 0.28281828, 0.28576312, 0.28761143, 0.28830464]), 'split1_test_score': array([-0.06383614, -0.071352  , -0.07470951, -0.07192148, -0.06998664]), 'split2_test_score': array([-0.16179865, -0.15862771, -0.15186788, -0.13136608, -0.11724562]), 'split3_test_score': array([-19.49129427, -19.65899689, -19.74151   , -19.67780931,
     

In [61]:
pd.DataFrame(results_grid['Until week: 0']['Lasso'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.301727,0.065017,0.007353,0.000978,0.1,{'alpha': 0.1},-2.545750,-0.080014,-37.885538,-2.129004,0.197399,-8.488581,14.738336,5
1,2.356404,0.127121,0.007363,0.000510,0.2,{'alpha': 0.2},-2.488585,-0.092630,-28.968431,-1.979790,0.200169,-6.665853,11.199672,4
2,2.250273,0.034085,0.008317,0.001236,0.3,{'alpha': 0.3},-2.468070,-0.104030,-21.328622,-1.891113,0.196326,-5.119102,8.168245,3
3,2.289234,0.081345,0.008160,0.000728,0.4,{'alpha': 0.4},-2.464836,-0.099678,-14.930238,-1.833980,0.191584,-3.827430,5.641834,2
4,2.380000,0.112700,0.008777,0.000745,0.5,{'alpha': 0.5},-2.451951,-0.096971,-9.788240,-1.773654,0.187884,-2.784586,3.639853,1


In [62]:
metrics_grid_lasso = pd.DataFrame(columns= ['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_alpha', 'params', 'split0_test_score', 'split1_test_score',
       'split2_test_score', 'split3_test_score', 'split4_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'])
for key in results_grid:
    df = pd.DataFrame(results_grid[key]['Lasso'])
    metrics_grid_lasso= metrics_grid_lasso.append(df)

In [63]:
metrics_grid_lasso

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.301727,0.065017,0.007353,0.000978,0.1,{'alpha': 0.1},-2.545750,-0.080014,-37.885538,-2.129004,0.197399,-8.488581,14.738336,5
1,2.356404,0.127121,0.007363,0.000510,0.2,{'alpha': 0.2},-2.488585,-0.092630,-28.968431,-1.979790,0.200169,-6.665853,11.199672,4
2,2.250273,0.034085,0.008317,0.001236,0.3,{'alpha': 0.3},-2.468070,-0.104030,-21.328622,-1.891113,0.196326,-5.119102,8.168245,3
3,2.289234,0.081345,0.008160,0.000728,0.4,{'alpha': 0.4},-2.464836,-0.099678,-14.930238,-1.833980,0.191584,-3.827430,5.641834,2
4,2.380000,0.112700,0.008777,0.000745,0.5,{'alpha': 0.5},-2.451951,-0.096971,-9.788240,-1.773654,0.187884,-2.784586,3.639853,1
0,2.445464,0.111383,0.008626,0.001015,0.1,{'alpha': 0.1},-2.046427,0.035139,-17.776466,-3.266531,0.209030,-4.569051,6.730893,5
1,2.428812,0.010073,0.009010,0.001039,0.2,{'alpha': 0.2},-2.038451,0.026508,-11.264322,-3.223088,0.209716,-3.257927,4.204825,4
2,2.495981,0.040120,0.009034,0.000900,0.3,{'alpha': 0.3},-2.029100,0.021925,-6.327128,-3.200407,0.210077,-2.264926,2.399481,3
3,2.495871,0.034808,0.008002,0.001072,0.4,{'alpha': 0.4},-2.020605,0.023065,-2.958567,-3.202634,0.210809,-1.589587,1.449405,2
4,2.598715,0.075030,0.009777,0.001161,0.5,{'alpha': 0.5},-2.021277,0.024080,-1.165302,-3.191947,0.212464,-1.228396,1.275378,1


In [ ]:
from sklearn import linear_model
df_train, df_test = pipeline.split_and_scale_on_last_weeks(df, n_weeks_prediction)
X_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
X_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')
lreg = linear_model.LinearRegression()
lreg.fit(X_train, y_train)
y_pred = lreg.predict(X_test)

In [64]:
pipeline.plot_real_vs_prediction(X_test, y_pred, y_test, 'Portugal')

NameError: name 'y_pred' is not defined